This notebook is used to analyse the simulated data of the deconvolution tests in larsoft. The output of the workflow is a collection of hits. The hits are then converted into a format that can be compared against the true photon information.

In [1]:
from lib import *
import numpy as np

import os
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from scipy.optimize import curve_fit

from plotly.subplots import make_subplots
from itertools import product

*** THIS NOTEBOOK CAN BE USED TO ANALYSE RAW AND DEC OP.WVFs FROM LARSOFT ***

In [2]:
deco_path = "data/deconv_reco_hist.root"
raw_path  = "data/opdetraw_hist.root"

data1 = load_larsoft_root("RAW","data/opdetraw_hist.root",     ["opdigi","opdigiana"],np.loadtxt("template/fbk_deco.txt"),label="RAW")
data2 = load_larsoft_root("DEC","data/deconv_gauss_hist.root", ["opdigi","opdecoana"],np.loadtxt("template/fbk_deco.txt"),label="GAUSS")
data3 = load_larsoft_root("DEC","data/deconv_wiener_hist.root",["opdigi","opdecoana"],np.loadtxt("template/fbk_deco.txt"),label="WIENER")

color_map={data1["LABEL"]: "#3366CC", data2["LABEL"]: "#66AA00", data3["LABEL"]:"#FF9900"}


In [3]:
raw_reco1 = ophit_data("data/ophitspe_gauss_hist.root", scaling=100,label="RAW")
reco1 = order_ophit_data(raw_reco1,data1["RECO"]["CH"])
raw_reco2 = ophit_data("data/ophitspe_gauss_hist.root", scaling=100,label="GAUSS")
reco2 = order_ophit_data(raw_reco2,data2["RECO"]["CH"])
raw_reco3 = ophit_data("data/ophitspe_wiener_hist.root",scaling=100,label="WIENER")
reco3 = order_ophit_data(raw_reco3,data3["RECO"]["CH"])

ophit_1 = combine_ophit_with_data(data1,reco1)
ophit_2 = combine_ophit_with_data(data2,reco2)
ophit_3 = combine_ophit_with_data(data3,reco3)

['EventID', 'HitID', 'OpChannel', 'PeakTimeAbs', 'PeakTime', 'Frame', 'Width', 'Area', 'Amplitude', 'PE', 'FastToTotal']
['EventID', 'HitID', 'OpChannel', 'PeakTimeAbs', 'PeakTime', 'Frame', 'Width', 'Area', 'Amplitude', 'PE', 'FastToTotal']
['EventID', 'HitID', 'OpChannel', 'PeakTimeAbs', 'PeakTime', 'Frame', 'Width', 'Area', 'Amplitude', 'PE', 'FastToTotal']


*** VISUALIZE DATA ***

In [21]:
# Choose between a random channel number or select yourself
# ch = 218
ch = np.random.choice(data1["RECO"]["CH"])
num = np.where(np.asarray(data2["RECO"]["CH"]) == ch)[0][0]
# print(data1["TRUE"]["PETIMES"][num])
fig = make_subplots(rows=1, cols=3,subplot_titles=('', ''))

fig.add_trace(go.Scatter(line=dict(color=color_map[data1["LABEL"]]),name="%s #PE: %.2f"%(data1["LABEL"],data1["RECO"]["PE"][num]),x=data1["RECO"]["WVF_X"][num],y=data1["RECO"]["WVF"][num]-data1["PEDESTAL"]),col=1,row=1)
fig.add_trace(go.Scatter(line=dict(color=color_map[data2["LABEL"]]),name="%s #PE: %.2f"%(data2["LABEL"],data2["RECO"]["PE"][num]),x=data2["RECO"]["WVF_X"][num],y=data2["RECO"]["WVF"][num]),col=2,row=1)
fig.add_trace(go.Scatter(line=dict(color=color_map[data3["LABEL"]]),name="%s #PE: %.2f"%(data3["LABEL"],data3["RECO"]["PE"][num]),x=data3["RECO"]["WVF_X"][num],y=data3["RECO"]["WVF"][num]),col=3,row=1)

fig.add_vline(x=data1["RECO"]["T0"][num], line_width=2, line_dash="dash", line_color="gray",col=1)
fig.add_vline(x=data2["RECO"]["T0"][num], line_width=2, line_dash="dash", line_color="gray",col=2)
fig.add_vline(x=data3["RECO"]["T0"][num], line_width=2, line_dash="dash", line_color="gray",col=3)

try:
    if len(data1["TRUE"]["PETIMES"][num]) > 1000:
        fig.add_trace(go.Histogram(name="TRUE #PE: %.2f"%(len(data1["TRUE"]["PETIMES"][num])),x=np.asarray(data1["TRUE"]["PETIMES"][num])+data1["PRETRIGGER"]*data1["SAMPLING"]*1e6),col=1,row=1)
    else:
        fig.add_trace(go.Scatter(marker_symbol="triangle-up",mode="markers",line=dict(color="black"),name="TRUE #PE: %.2f"%(len(data1["TRUE"]["PETIMES"][num])),x=np.asarray(data1["TRUE"]["PETIMES"][num]),y=np.zeros(len(data1["TRUE"]["PETIMES"][num]))),col=1,row=1)
        fig.add_trace(go.Scatter(marker_symbol="triangle-up",mode="markers",line=dict(color="black"),name="TRUE #PE: %.2f"%(len(data1["TRUE"]["PETIMES"][num])),x=np.asarray(data1["TRUE"]["PETIMES"][num]),y=np.zeros(len(data1["TRUE"]["PETIMES"][num]))),col=2,row=1)
        fig.add_trace(go.Scatter(marker_symbol="triangle-up",mode="markers",line=dict(color="black"),name="TRUE #PE: %.2f"%(len(data1["TRUE"]["PETIMES"][num])),x=np.asarray(data1["TRUE"]["PETIMES"][num]),y=np.zeros(len(data1["TRUE"]["PETIMES"][num]))),col=3,row=1)
except:
    print("WVF WITHOUT TRUE PHOTONS")

# fig.add_trace(go.Scatter(name="OPHITFINDER #PE: %.2f"%(np.sum(reco1[ch]["PE"])),line=dict(color="#DC3912"),mode="markers",x=reco1[ch]["PeakTime"],y=reco1[ch]["Amplitude"]),col=1,row=1)
fig.add_trace(go.Scatter(name="OPHITFINDER #PE: %.2f"%(ophit_2["PE"][num]),line=dict(color="#DC3912"),mode="markers",x=ophit_2["TIMES"][num],y=ophit_2["AMP"][num]/reco2["SCALING"]),col=2,row=1)
fig.add_trace(go.Scatter(name="OPHITFINDER #PE: %.2f"%(ophit_3["PE"][num]),line=dict(color="#DC3912"),mode="markers",x=ophit_3["TIMES"][num],y=ophit_3["AMP"][num]/reco3["SCALING"]),col=3,row=1)


fig.update_layout(title="Deco. wvf comparison for ch %i"%ch,xaxis_title="Time in [&mu;s]",yaxis_title="Amp. in [ADC]",xaxis2_title="Time in [&mu;s]",yaxis2_title="Amp. in [a.u.]",xaxis3_title="Time in [&mu;s]",yaxis3_title="Amp. in [a.u.]")
fig.update_layout(autosize=True,font=dict(size=14))

fig.show()

*** CONFIGURE FILTER FOR ANALYSIS ***

In [5]:


pe_error_1 = (np.asarray(data1["RECO"]["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
pe_error_2 = (np.asarray(data2["RECO"]["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
pe_error_3 = (np.asarray(data3["RECO"]["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])

ophitpe_error_1 = (np.asarray(ophit_1["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
ophitpe_error_2 = (np.asarray(ophit_2["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
ophitpe_error_3 = (np.asarray(ophit_3["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])

t0_reco_1 = np.asarray(data1["RECO"]["T0"])-np.asarray(data1["TRUE"]["T0"])
t0_reco_2 = np.asarray(data2["RECO"]["T0"])-np.asarray(data1["TRUE"]["T0"])
t0_reco_3 = np.asarray(data3["RECO"]["T0"])-np.asarray(data1["TRUE"]["T0"])

amp           = np.concatenate([data1["RECO"]["AMP"],data2["RECO"]["AMP"],data3["RECO"]["AMP"]])
pe_reco       = np.concatenate([data1["RECO"]["PE"],data2["RECO"]["PE"],data3["RECO"]["PE"]])
pe_true       = np.concatenate([data1["TRUE"]["PE"],data1["TRUE"]["PE"],data1["TRUE"]["PE"]])
pe_ophit      = np.concatenate([ophit_1["PE"],ophit_2["PE"],ophit_3["PE"]])
ophitpe_error = 100*np.concatenate([ophitpe_error_1,ophitpe_error_2,ophitpe_error_3])
pe_error      = 100*np.concatenate([pe_error_1,pe_error_2,pe_error_3])
t0_reco       = np.concatenate([t0_reco_1,t0_reco_2,t0_reco_3])
filter_label  = np.concatenate([[data1["LABEL"]]*len(pe_error_1),[data2["LABEL"]]*len(pe_error_2),[data3["LABEL"]]*len(pe_error_3)])

df = pd.DataFrame({"FILTER":filter_label, "'%'ERROR PE":pe_error, "'%'ERROR OPHIT PE":ophitpe_error, "TRUE PE":pe_true, "RECO PE":pe_reco, "OPHIT PE":pe_ophit, "AMP [a.u.]":amp, "RECO T0":t0_reco})

/tmp/ipykernel_159881/44604923.py:1: RuntimeWarning:

divide by zero encountered in divide

/tmp/ipykernel_159881/44604923.py:2: RuntimeWarning:

divide by zero encountered in divide

/tmp/ipykernel_159881/44604923.py:3: RuntimeWarning:

divide by zero encountered in divide

/tmp/ipykernel_159881/44604923.py:5: RuntimeWarning:

divide by zero encountered in divide

/tmp/ipykernel_159881/44604923.py:5: RuntimeWarning:

invalid value encountered in divide

/tmp/ipykernel_159881/44604923.py:6: RuntimeWarning:

divide by zero encountered in divide

/tmp/ipykernel_159881/44604923.py:6: RuntimeWarning:

invalid value encountered in divide

/tmp/ipykernel_159881/44604923.py:7: RuntimeWarning:

divide by zero encountered in divide

/tmp/ipykernel_159881/44604923.py:7: RuntimeWarning:

invalid value encountered in divide



In [6]:
fig = px.scatter(data_frame=df[(df['TRUE PE'] > 10)*(df['TRUE PE'] < 1000)], y='TRUE PE', x="AMP [a.u.]",
color='FILTER',color_discrete_map=color_map, marginal_x='histogram',trendline="ols")
fig.update_layout(autosize=False,width=1000,height=800,font=dict(size=16))
fig.show()

results = px.get_trendline_results(fig)
# print(results)
results.query("FILTER == 'GAUSS'").px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.993
Method:                 Least Squares   F-statistic:                 6.215e+04
Date:                Wed, 29 Mar 2023   Prob (F-statistic):               0.00
Time:                        18:32:32   Log-Likelihood:                -2102.9
No. Observations:                 462   AIC:                             4210.
Df Residuals:                     460   BIC:                             4218.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         11.8683      1.402      8.466      0.000       9.113      14.623
x1            25.6330      0.103    249.291      0.000      25.431      25.835
==============================================================================
Omnibus:                       64.115   Durbin-Watson:                   1.900
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              328.358
Skew:                           0.454   Prob(JB):                     4.99e-72
Kurtosis:                       7.029   Cond. No.                         17.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
fig = px.scatter(data_frame=df[(df['TRUE PE'] > 10)*(df['TRUE PE'] < 2000)], y="TRUE PE", x="'%'ERROR OPHIT PE",
color='FILTER',color_discrete_map=color_map,marginal_x='box',template="plotly")
# fig.update_layout(title_text='T0 recovery',xaxis_title  = "Time in [&mu;s]",yaxis_title  = "Counts")
fig.update_layout(autosize=False,width=1500,height=800,font=dict(size=16))
fig.update_traces(opacity=0.75)
fig.show()

In [8]:
fig = px.histogram(data_frame=df[(df['TRUE PE'] > 0)*(df['TRUE PE'] < 1000)], x="RECO T0",
color='FILTER',color_discrete_map=color_map,barmode="group",marginal='box',template="plotly",nbins=1000,range_x=[-0.16,0.32])
# fig.update_layout(title_text='T0 recovery',xaxis_title  = "Time in [&mu;s]",yaxis_title  = "Counts")
fig.update_layout(autosize=False,width=1500,height=800,font=dict(size=16))

fig.show()

In [9]:
min_wvf_length = 500
ave_wvfs = []
for data in [data2,data3]:
    data["RECO"]["SHORT_WVF"] = []
    for wvf in data["RECO"]["WVF"]:
        if len(wvf) > min_wvf_length:
            data["RECO"]["SHORT_WVF"].append(wvf[:min_wvf_length])
    # ave_wvf = np.mean(np.asarray(data["RECO"]["SHORT_WVF"])[np.where(np.asarray(data1["TRUE"]["PE"]) > 200)[0].tolist()],axis=0)
    ave_wvf = np.mean(np.asarray(data["RECO"]["SHORT_WVF"]),axis=0)
    ave_wvf = ave_wvf/np.max(ave_wvf)
    ave_wvf = ave_wvf[np.argmax(ave_wvf):]
    ave_wvfs.append(ave_wvf)
    initial = [0.7,0.3,6,1400]
    labels = ["CONSTANT","AMPLITUDE","TAU FAST","TAU SLOW"]
    popt, pcov = curve_fit(scint_profile,16*np.arange(len(ave_wvf)),ave_wvf,p0=initial)
    perr = np.sqrt(np.diag(pcov))

    ave_wvfs.append(scint_profile(16*np.arange(len(ave_wvf)),*popt))
    print("\n----------- FIT VALUES " + data["LABEL"]+ " ------------")
    for i in range(len(initial)):
        print("%s:\t%.2E\t%.2E"%(labels[i], popt[i], perr[i]))
    print("-----------------------------------------")

wvfs  = np.concatenate([ave_wvfs[0],ave_wvfs[1],ave_wvfs[2],ave_wvfs[3]])
wvfs_x  = np.concatenate([16*np.arange(len(ave_wvf)),16*np.arange(len(ave_wvf)),16*np.arange(len(ave_wvf)),16*np.arange(len(ave_wvf))])
fit_label  = np.concatenate([["WVF"]*len(ave_wvf),["FIT"]*len(ave_wvf),["WVF"]*len(ave_wvf),["FIT"]*len(ave_wvf)])
filter_label  = np.concatenate([[data2["LABEL"]]*len(ave_wvf),[data2["LABEL"]]*len(ave_wvf),[data3["LABEL"]]*len(ave_wvf),[data3["LABEL"]]*len(ave_wvf)])

fit_df = pd.DataFrame({"WVF":wvfs,"TIME in [ns]":wvfs_x,"FILTER":filter_label,"FIT":fit_label})

fig = px.line(data_frame=fit_df,x="TIME in [ns]",y="WVF",color="FIT",facet_col="FILTER",log_y=True,color_discrete_map=color_map)
fig.show()


----------- FIT VALUES GAUSS ------------
CONSTANT:	1.83E+02	1.95E+00
AMPLITUDE:	1.82E-01	3.16E-03
TAU FAST:	7.10E+01	1.25E+00
TAU SLOW:	1.70E+03	3.63E+01
-----------------------------------------

----------- FIT VALUES WIENER ------------
CONSTANT:	1.89E+02	1.98E+00
AMPLITUDE:	1.77E-01	2.97E-03
TAU FAST:	7.10E+01	1.20E+00
TAU SLOW:	1.82E+03	3.78E+01
-----------------------------------------
